In [11]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from scipy.integrate import odeint
import pprint
from ipywidgets import interact, interactive, fixed, interact_manual, jslink
import ipywidgets as widgets
import ipyvolume as ipv
import ipyvolume.pylab as p3
import traitlets
import time, sys
from IPython.core.display import clear_output

In [12]:
# define the number of floating atoms
floaters = np.zeros(15*3) # set number of floaters and multiply by 3 to have 3 coords for each
n = len(floaters)
m = n/3                    # total atoms in the chain

# set the positions of the floating atoms
count = 0
k = 1.7
for i in range(0,n,3):
    floaters[i] = count
    floaters[i+1] = k*count
    floaters[i+2] = 1
    count += 1

In [13]:
# set number of substrate atoms (substrate will be 2*ns + 1 side lengths)
ns = 5

In [14]:
# set the substrate parameters
h_x = 1
h_y = 1

In [15]:
# set the VDW parameters
w = 1
sigma = 1

In [16]:
# set the spring parameters
k_s = 5
l = 1

In [17]:
# set the bending parameter
beta = 200

In [25]:
# define the time interval for the gradient flow
t = np.linspace(0,60,20)

In [26]:
# this is the RHS of the system of ODEs solved by odeint. It returns the forces
# acting on each floater in the x, y and z directions
def all_forces(floaters, t):

    # compute number of atoms
    m = int(len(floaters)/3)
    n = int(len(floaters))

    # split the atoms into 1x3 arrays
    r = [[0 for i in range(3)] for j in range(m)]
    for i in range(m):
        r[i] = floaters[i*3:i*3+3]

    # initialize arrays for the spring forces, VDW forces and bending forces
    combined_spring_forces = np.zeros(n)
    combined_VDW_forces = np.zeros(n)
    VDW_force = np.zeros((m, 3))
    spring_force = np.zeros((m, 3))
    current_bending_force = np.zeros((int(m),3))
    combined_bending_forces = np.zeros(int(m))

    # define the size of the substrate surrounding the floaters
    a = np.arange(-ns,ns+1,1)

    # compute the distances of the floating atom from the substrate atoms
    # and the VDW forces acting on them as a result
    for j in range(len(a)):
        for i in range(len(a)):
            for k in range(m):

                # compute the offsets
                k_x = (r[k][0] + .5)%h_x
                k_y = (r[k][1] + .5)%h_y

                # compute dx, dy, dz and rhat
                dx = k_x - .5 + a[i]*h_x
                dy = k_y - .5 + a[j]*h_y
                dz = r[k][2]
                rhat = np.sqrt(dx**2 + dy**2 + dz**2)
                gradV_common = (12*w*((sigma**6)/(rhat**8)-(sigma**12)/(rhat**14)))

                # this is the gradient of V (computed by hand), it is used to compute the
                # VDW forces acting on the floating atoms
                VDW_force[k] = -gradV_common*np.array([dx, dy, dz])

            # combine the VDW forces for all floating atoms
            combined_VDW_forces += np.hstack(VDW_force)

    # this is the gradient of E_s (computed by hand), it is used to compute the
    # spring forces acting on the atoms (m = number of atoms in the chain)
    for i in range(m-1):
        gradE_common = k_s*(norm(r[i]-r[i+1]) - l)/norm(r[i] - r[i+1])
        current_sf = -gradE_common*np.array([(r[i][0]-r[i+1][0]),(r[i][1]-r[i+1][1]),(r[i][2]-r[i+1][2])])
        spring_force[i] += current_sf
        spring_force[i+1] = -current_sf
    
    # loop for computing bending force
    for i in range(1,int(m-1)):          # loop through each atom not on the end of the chain and compute the bending force on it and its neighbors
        
        # define the pieces of the bending energy equation
        sx_n = floaters[i*3] - floaters[(i-1)*3]
        sx_n1 = floaters[(i+1)*3] - floaters[i*3]

        sy_n = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        sy_n1 = floaters[(i+1)*3 + 1] - floaters[i*3 + 1]

        sz_n = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        sz_n1 = floaters[(i+1)*3 + 2] - floaters[i*3 + 2]

        A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

        B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

        partial_common = .5/A
        
        # partials of A wrt x for atom n, n-1 and n+1, respectively
        Axn = partial_common*(2*(sx_n)*(sx_n1**2) - 2*(sx_n1)*(sx_n**2) + 2*(sx_n)*(sy_n1**2) + 2*(sx_n)*(sz_n1**2) - 2*(sx_n1)*(sy_n**2) - 2*(sx_n1)*(sz_n**2))
        Axn0 = partial_common*(-2*(sx_n)*(sx_n1**2) - 2*(sx_n)*(sy_n1**2) - 2*(sx_n)*(sz_n1**2))
        Axn1 = partial_common*(2*(sx_n**2)*(sx_n1) + 2*(sy_n**2)*(sx_n1) + 2*(sz_n**2)*(sx_n1))
        
        # partials of A wrt y for atom n, n-1 and n+1, respectively
        Ayn = partial_common*(-2*(sx_n**2)*(sy_n1) + 2*(sy_n)*(sx_n1**2) + 2*(sy_n)*(sy_n1**2) - 2*(sy_n**2)*(sy_n1) + 2*(sy_n)*(sz_n1**2) - 2*(sz_n**2)*(sy_n1))
        Ayn0 = partial_common*(-2*(sy_n)*(sx_n1**2) - 2*(sy_n)*(sy_n1**2) - 2*(sy_n)*(sz_n1**2))
        Ayn1 = partial_common*(2*(sy_n**2)*(sy_n1) + 2*(sz_n**2)*(sy_n1) + 2*(sx_n**2)*(sy_n1))
        
        # partials of A wrt z for atom n, n-1 and n+1, respectively
        Azn = partial_common*(-2*(sx_n**2)*(sz_n1) - 2*(sy_n**2)*(sz_n1) - 2*(sz_n**2)*(sz_n1) + 2*(sz_n)*(sz_n1**2) + 2*(sz_n)*(sx_n1**2) + 2*(sz_n)*(sy_n1**2))
        Azn0 = partial_common*(-2*(sz_n)*(sx_n1**2) -2*(sz_n)*(sy_n1**2) - 2*(sz_n)*(sz_n1**2))
        Azn1 = partial_common*(2*(sz_n1)*(sx_n**2) + 2*(sz_n1)*(sy_n**2) + 2*(sz_n1)*(sz_n**2))
        
        # partials of B wrt x for atom n, n-1 and n+1, respectively
        Bxn = floaters[(i+1)*3] - 2*floaters[i*3] + floaters[(i-1)*3]
        Bxn0 = -floaters[(i+1)*3] + floaters[i*3]
        Bxn1 = floaters[i*3] - floaters[(i-1)*3]

        # partials of B wrt y for atom n, n-1 and n+1, respectively
        Byn = floaters[(i+1)*3 + 1] - 2*floaters[i*3 + 1] + floaters[(i-1)*3 + 1]
        Byn0 = -floaters[(i+1)*3 + 1] + floaters[i*3 + 1]
        Byn1 = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        
        # partials of B wrt z for atom n, n-1 and n+1, respectively
        Bzn = floaters[(i+1)*3 + 2] - 2*floaters[i*3 + 2] + floaters[(i-1)*3 + 2]
        Bzn0 = -floaters[(i+1)*3 + 2] + floaters[i*3 + 2]
        Bzn1 = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        
        try:
            Eb_common = 4*beta/((A+B)**2)                   # compute common part of bending energy gradient
            if (np.isnan(Eb_common)  or np.isinf(Eb_common) or np.isnan(A)):
                raise ValueError('Divide by zero')
                raise Exception
            
        except Exception as error:
            print("Caught exception: ", error)
            continue
            
        current_bending_force[i,0] += Eb_common*(A*Bxn - B*Axn)
        current_bending_force[i,1] += Eb_common*(A*Byn - B*Ayn)
        current_bending_force[i,2] += Eb_common*(A*Bzn - B*Azn)
            
        current_bending_force[i-1,0] += Eb_common*(A*Bxn0 - B*Axn0)
        current_bending_force[i-1,1] += Eb_common*(A*Byn0 - B*Ayn0)
        current_bending_force[i-1,2] += Eb_common*(A*Bzn0 - B*Azn0)
            
        current_bending_force[i+1,0] = Eb_common*(A*Bxn1 - B*Axn1)
        current_bending_force[i+1,1] = Eb_common*(A*Byn1 - B*Ayn1)
        current_bending_force[i+1,2] = Eb_common*(A*Bzn1 - B*Azn1)

    # combine all of the spring and bending forces into arrays, then merge all of the force arrays into a total force array
    combined_bending_forces = np.hstack(current_bending_force)
    combined_spring_forces = np.hstack(spring_force)
    forces = np.add(combined_spring_forces, combined_VDW_forces, combined_bending_forces)

    return forces


In [27]:
%%time
# compute the changing positions of the atoms as a result of the spring
# and VDW forces acting on them
gFlow = odeint(all_forces, floaters, t, atol=1.4e-10)

CPU times: user 26.9 s, sys: 444 ms, total: 27.3 s
Wall time: 28.1 s


/anaconda/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


# Simulation Code

In [28]:
# define arrays of x, y and z coordinates to create an animated scatter plot
x = np.zeros((len(t),int(m)))
y = np.zeros((len(t),int(m)))
z = np.zeros((len(t),int(m)))
for i in range(len(t)):
    for j in range(int(m)):
        x[i,j] = gFlow[i,j*3]
        y[i,j] = gFlow[i,j*3+1]
        z[i,j] = gFlow[i,j*3+2]

In [29]:
%matplotlib inline

# create an ipyvolume.pylab figure for the simulation
p3.figure()

# plot a scatterplot of the arrays defined above in the figure
s1 = p3.scatter(x, y, z, marker='sphere', connected=True)
s2 = p3.plot(x, y, z)

# set the x, y and z limits of the figure
p3.xlim(-50,120)
p3.ylim(-50,120)
p3.zlim(0,1)

# add animation controls to the figure
p3.animation_control([s1, s2], interval=400)

# add sliders for parameters
ns_slider = widgets.IntSlider(min=0, max=15, step=1, value=ns, description='ns', continuous_update=False)
w_slider = widgets.IntSlider(min=0, max=10, step=1, value=w, description='w', continuous_update=False)
sigma_slider = widgets.IntSlider(min=0, max=10, step=1, value=sigma, description='sigma', continuous_update=False)
k_s_text = widgets.FloatText(min=0., max=500., step=.1, value=k_s, description='k_s', continuous_update=False)
l_slider = widgets.IntSlider(min=0, max=10, step=1, value=l, description='l', continuous_update=False)
beta_slider = widgets.IntSlider(min=0, max=15, step=1, value=beta, description='beta', continuous_update=False)

# create handler functions to change the values of the parameters globally when the sliders change
def on_ns_slider_change(change):
    global ns
    ns = change.new

def on_w_slider_change(change):
    global w
    w = change.new

def on_sigma_slider_change(change):
    global sigma
    sigma = sigma.new
    
def on_k_s_text_change(change):
    global k_s
    k_s = change.new
    
def on_l_slider_change(change):
    global l
    l = change.new

def on_beta_slider_change(change):
    global beta
    beta = change.new


# set the sliders to call their handler functions any time the sliders are moved
ns_slider.observe(on_ns_slider_change, names='value')
w_slider.observe(on_w_slider_change, names='value')
sigma_slider.observe(on_sigma_slider_change, names='value')
k_s_text.observe(on_k_s_text_change, names='value')
l_slider.observe(on_l_slider_change, names='value')
beta_slider.observe(on_beta_slider_change, names='value')

# create a plot of the total system energy over time
#f, ax = plt.subplots()

#for i in range(len(t)):
    
    # set axes labels for the plot
#    plt.xlabel('Time')
#    plt.ylabel('Total System Energy')
    
    # plot the line and the current point on the line
#    ax.plot(t, total_system_energy, 'k')
#    ax.plot(t[i], total_system_energy[i], 'ro')
    
    # pause
    #time.sleep(0.1)
    
    # clear output, otherwise, we end up with len(t) graphs
    #clear_output()
    
#    display(f)
#    ax.cla()

# display the VBox beneath the total system energy plot
label = widgets.Label('Parameter sliders:')
parameters = widgets.VBox([ipv.gcc(), label, ns_slider, w_slider, sigma_slider, l_slider, k_s_text])
display(parameters)

VBox(children=(VBox(children=(Figure(animation=400.0, animation_exponent=1.0, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white',
      dtype='<U5'), connected=True, geo='sphere', size=array(2), size_selected=array(2.6), x=array([[  0.00000000e+000,   1.00000000e+000,   2.00000000e+000,
          3.00000000e+000,   4.00000000e+000,   5.00000000e+000,
          6.00000000e+000,   7.00000000e+000,   8.00000000e+000,
          9.00000000e+000,   1.00000000e+001,   1.10000000e+001,
          1.20000000e+001,   1.30000000e+001,   1.40000000e+001],
       [  5.20571698e-001,   1.51575466e+000,   2.50016563e+000,
          3.48701400e+000,   4.51161983e+000,   5.46804398e+000,
          5.59004253e+000,   7.46705565e+000,   8.46361006e+000,
          8.52499494e+000,   9.50954336e+000,   1.05024051e+001,
          1.14870796e+001,   1.25106034e+001,   1.34628392e+001],
       [ -6.51894584e+296,   6.56290069e+136,  -2.11855543e-269,
          7.43808068e+149,  -1.30414126e+273,   6.10285006e+241,
         -2.50233256e-113,  -1.09685053e+249,   1.02765845e-053,
          1.41836909e-026,   7.41896052e-186,  -3.66455910e+061,
         -1.10279216e+152,   1.64439656e-234,   3.88304375e+164],
       [ -7.18584138e-001,  -7.31027426e+003,   6.89118153e-087,
         -2.66012722e-210,   4.15403381e+196,   2.24202445e-302,
          3.56286310e-033,   7.29528695e-212,  -6.58346911e-035,
         -1.87483616e-168,   1.05231229e+157,  -4.73344591e-009,
          5.39527147e-278,  -8.46709548e-017,   9.48000595e-086],
       [  2.29076116e+181,  -5.05727081e-086,   6.38851454e-236,
          2.32052758e-038,   6.58815571e+150,  -2.62474287e-304,
          8.25473683e+169,  -9.70215654e-084,   2.98356569e-168,
         -5.93545581e+233,   4.49132966e+067,   3.47495960e-264,
         -2.17818336e-046,   2.11730005e-138,   1.24141304e+163],
       [  1.03055758e-222,   4.36653473e-191,   5.25185622e-198,
          2.12445253e-172,  -1.47255597e-217,  -2.54258191e+063,
         -8.03167471e+144,  -1.10468359e-210,  -3.25039066e+038,
         -2.38393690e+121,   1.13323486e-265,  -2.35043563e-269,
         -6.52224098e+278,   1.92470024e-010,  -5.17455345e+251],
       [ -2.11079458e+158,  -1.50832298e+249,   2.07814664e-265,
         -3.00306769e-027,   1.08318316e-031,   1.61757723e+021,
          1.70494299e+154,  -9.67049860e-143,  -4.68151469e-119,
         -3.47309287e+124,  -2.08143670e+259,  -6.95108041e-276,
         -1.53682333e-129,  -6.35401720e+271,   3.11422275e-147],
       [ -1.15642801e+055,  -2.60573262e+071,   4.07752462e-128,
          1.07328590e-191,  -1.54645902e-065,   4.29069171e-196,
          5.31314576e+035,  -1.09705756e+194,  -1.67252906e-184,
         -1.90663139e+058,   8.33212420e-290,  -9.52462246e-217,
         -1.25527930e+227,   2.00671765e+017,  -1.07601626e+207],
       [ -8.87975414e+136,   3.31683928e+287,  -1.28599522e-028,
          7.44144354e-299,   6.08906934e+237,   2.03529728e+284,
         -1.24830889e+057,   4.25405475e-064,   5.29326365e+183,
         -6.35755914e-243,  -1.10816143e+077,   7.98686543e+079,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+0